In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import time
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense,Dropout
from keras import Sequential
from sklearn.metrics import classification_report
from keras.layers import Conv2D,Flatten,MaxPool2D,BatchNormalization,Activation,Dropout
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
import cv2
from PIL import Image
import csv
import mediapipe as mp
from keras.models import load_model

In [2]:
#model_path = r'C:/Users/Maxime.ALTER/ProjetIA/ProjetIA/Dev_max/CNN.pb'
#model2 = tf.saved_model.load(model_path)
#model3 = pickle.load()

In [3]:
df = pd.read_csv("landmark.csv",on_bad_lines='skip')
X = df.drop(df.columns[0], axis = 1)
y = df[df.columns[0]]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

len(y_train.unique())

#y_train=keras.utils.to_categorical(y_train,27)
#y_test=keras.utils.to_categorical(y_test)

24

In [5]:
model1 = tf.keras.Sequential([
    tf.keras.layers.Dense(units=63, input_shape=[63]),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dense(units=512, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=256, activation='relu'),
    tf.keras.layers.Dense(units=128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=64, activation='relu'),
    tf.keras.layers.Dense(units=24, activation='softmax')
])

model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 63)                4032      
                                                                 
 dropout (Dropout)           (None, 63)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8192      
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                                 
 dense_3 (Dense)             (None, 512)               131584    
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dropout_1 (Dropout)         (None, 512)               0

In [6]:
y_train_encoded = pd.get_dummies(y_train, prefix='target')
y_test_encoded = pd.get_dummies(y_test, prefix='target')
y_train_encoded.head(3)

,target_0,target_1,target_2,target_3,target_4,target_5,target_6,target_7,target_8,target_9,...,target_14,target_15,target_16,target_17,target_18,target_19,target_20,target_21,target_22,target_23
52492,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7305,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29994,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=10,
    restore_best_weights=True
)

In [8]:
model1.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [9]:
start = time.time()

history = model1.fit(
    X_train,
    y_train_encoded,
    validation_split =0.2,
    epochs=5,
    callbacks=[early_stop]
)

stop1 = time.time()


y_pred = model1.predict(X_test)

stop2 = time.time()
apprentissage_RFC = round(stop1-start,3)
prediction_RFC = round(stop2-start,3)
print(f"Temps d'apprentissage : {apprentissage_RFC} s")
print(f"Temps d'apprentissage+prediction : {prediction_RFC} s")

Epoch 1/5
1082/1082 [==============================] - 10s 8ms/step - loss: 0.9018 - accuracy: 0.6861 - val_loss: 0.2601 - val_accuracy: 0.9101
Epoch 2/5
1082/1082 [==============================] - 10s 9ms/step - loss: 0.3314 - accuracy: 0.8864 - val_loss: 0.2302 - val_accuracy: 0.9247
Epoch 3/5
1082/1082 [==============================] - 9s 8ms/step - loss: 0.2647 - accuracy: 0.9153 - val_loss: 0.1855 - val_accuracy: 0.9391
Epoch 4/5
1082/1082 [==============================] - 9s 8ms/step - loss: 0.2213 - accuracy: 0.9312 - val_loss: 0.1114 - val_accuracy: 0.9674
Epoch 5/5
451/451 [==============================] - 1s 3ms/step
Temps d'apprentissage : 46.657 s
Temps d'apprentissage+prediction : 48.521 s


In [10]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [11]:
liste_lettre=['A','B','C','D','E','F','G','H','I','K','L','M','N','O','P','Q','R','S','T','U','V','W',"X","Y"]
liste_nb = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]

# Create a dictionary that maps letters to numbers
key = {nb: lettre for nb, lettre in zip(liste_nb, liste_lettre)}

# Print the dictionary
print(key)

{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'K', 10: 'L', 11: 'M', 12: 'N', 13: 'O', 14: 'P', 15: 'Q', 16: 'R', 17: 'S', 18: 'T', 19: 'U', 20: 'V', 21: 'W', 22: 'X', 23: 'Y'}


In [12]:
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
   

    # Define the font to use
    font = cv2.FONT_HERSHEY_PLAIN

    # Define the font scale
    scale = 3.0

    # Define the color of the text (in BGR format)
    color = (255, 255, 255)

    # Define the coordinates of the bottom-left corner of the text
    x_pos = 10
    y_pos = image.shape[0] - 10

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
        for hand_no, hand_landmarks in enumerate(results.multi_hand_landmarks):
            coord = []
            for i in range(21):
                
                x = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].x
                y = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].y
                z = hand_landmarks.landmark[mp_hands.HandLandmark(i).value].z
                coord.append([x, y, z])
            coord_rs = tf.reshape(coord, [1, -1])
            live_pred = model1.predict(coord_rs)
            live_pred = np.argmax(live_pred, axis=1)
            display = key.get(int(live_pred))
            # Define the text to display on the frame
            text = f"Prediction : {display}"
            
            
            # Call the putText method to draw the text on the frame
            image = cv2.putText(image, text, (int(x_pos), int(y_pos)), font, scale, color, thickness = 2)

            mp_drawing.draw_landmarks(
                image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
            
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', image)
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()

1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 23ms/step


In [16]:
print(display)


L


'L'

In [18]:
# Import the openpyxl module
import openpyxl
count = 1

# code = input("Entrez la lettre que vous souhaitez utilise comme mot de passe: ")
# Ask the user to input a number
while True:
  number = str(input("Entrez une lettre pour lancer le code: "))
  # Check if the number is 3
  if number == display:
    # If the number is 3, open the Excel file
    wb = openpyxl.load_workbook("file.xlsx")
    # Open the worksheet with the name "Sheet1"
#     worksheet = wb.get_sheet_by_name("Feuil1")
    worksheet = wb["Feuil1"]
    # Write the string to cell A1 in the worksheet, including the value of the number variable
    last_row = worksheet.max_row
    
    worksheet.cell(row=last_row+1,column=1).value = f"Vous avez utilise la lettre : {display} pour entrer dans la feuille excel, Il vous a fallu {count} essais"
    # Save the changes to the Excel file 
    wb.save("file.xlsx")
    break
  else:
    # If the number is not 3, print a message
    print("La lettre que vous avez rentré n'est pas le code.")
    count+=1

Entrez une lettre pour lancer le code: I
La lettre que vous avez rentré n'est pas le code.
Entrez une lettre pour lancer le code: A
La lettre que vous avez rentré n'est pas le code.
Entrez une lettre pour lancer le code: O
La lettre que vous avez rentré n'est pas le code.
Entrez une lettre pour lancer le code: P
La lettre que vous avez rentré n'est pas le code.
Entrez une lettre pour lancer le code: K
La lettre que vous avez rentré n'est pas le code.
Entrez une lettre pour lancer le code: L
